In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
al=pd.read_csv("../Ubuntu_files/Epoch_Hackathon/Dataset.csv")
# Train:test ratio = 80:20
# Therefore, I am taking 1280 training data and 319 test data
# Here we are going to take index 960 to 1279 as test data and rest as training data
x1=pd.concat([al[:640],al[959:]])

In [58]:
x1

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,k_value,l_value,m_value,percentage_free_sulphur,n_value
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,0,8.100,4.0500,0.636,30.909091,0.6080
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,0,8.680,4.3400,0.778,26.800000,0.8290
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,0,8.560,4.2800,0.742,36.000000,0.7440
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,1,11.480,5.7400,0.655,35.294118,0.7195
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,0,8.100,4.0500,0.636,30.909091,0.6080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,0,6.800,3.4000,0.670,13.750000,0.6610
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,1,6.450,3.2250,0.822,13.076923,0.7110
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,1,6.810,3.4050,0.826,13.793103,0.7540
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,0,6.545,3.2725,0.785,13.750000,0.6615


In [59]:
# Dropping k_value, l_value, m_value, n_value features of training data
x2=x1.drop(['k_value','l_value','m_value','n_value'],axis=1)

# Extracting quality of training data in numpy array "y"
y1=x2['quality']
y2=y1.to_numpy()
y=y2.reshape(1280,1)

# Removing quality from training data
x=x2.drop('quality',axis=1).to_numpy() 

# Test data from file
te1=al[960:1279]    

# Extracting quality of test data in numpy array "teans"
teans1=te1['quality']   
teans2=teans1.to_numpy()
teans=teans2.reshape(319,1)

# Since 'quality','k_value','l_value','m_value','n_value' are removed from training data, removing it from
# test data
te=te1.drop(['quality','k_value','l_value','m_value','n_value'],axis=1).to_numpy()

In [60]:
# Removing outliers from training data
temp1=np.percentile(x,1,axis=0)
temp99=np.percentile(x,99,axis=0)
c1=0
for col in range(12):
    p_1=temp1[col]
    p_99=temp99[col]
    for i in range(1280):
        if x[i][col]>p_99:
            x=np.delete(x,i,axis=0)
            y=np.delete(y,i,axis=0)
            c1+=1
        elif x[i][col]<p_1:
            x=np.delete(x,i,axis=0)
            y=np.delete(y,i,axis=0)
            c1+=1
        if (i==1279-c1):
            break

In [61]:
# Normalization of training data along with test data
n_row=x.shape[0]
for col in range(12):
    min_col=np.Infinity
    max_col=(-1)*np.Infinity
    for row in range(n_row):
        temp1=x[row][col]
        if (temp1>max_col):
            max_col=temp1
        elif (temp1<min_col):
            min_col=temp1
    x[:,col]=(x[:,col]-min_col)/(max_col-min_col)
    te[:,col]=(te[:,col]-min_col)/(max_col-min_col)

In [62]:
teans.shape

(319, 1)

In [63]:
def GDA_classif(x,y):
    """
    Return the list [Cov_mat,μ0,μ1,ϕ]
    x - mxn
    y - mx1
    """
    import numpy as np
    li=[]
    m,n=x.shape
    ϕ_sum=0
    μ0_sum=np.zeros(n)
    μ1_sum=np.zeros(n)
    μ0_sum=μ0_sum.reshape(n,1)
    μ1_sum=μ1_sum.reshape(n,1)
    μ0_count=0
    μ1_count=0
    for i in range(m):
        ϕ_sum+=y[i][0]
        if y[i][0]==0:
            μ0_sum+=x[i].reshape(n,1)
            μ0_count+=1
        elif y[i][0]==1:
            μ1_sum+=x[i].reshape(n,1)
            μ1_count+=1
    ϕ=ϕ_sum/m
    μ0=μ0_sum/μ0_count
    μ1=μ1_sum/μ1_count
    Covmat_sum=np.zeros(n*n)
    Covmat_sum=Covmat_sum.reshape(n,n)
    for i in range(m):
        if y[i][0]==0:
            temp=x[i].reshape(n,1)-μ0
        elif y[i][0]==1:
            temp=x[i].reshape(n,1)-μ1
        Covmat_sum+=temp@(temp.T)
    Cov_mat=Covmat_sum/m
    li.append(Cov_mat)
    li.append(μ0)
    li.append(μ1)
    li.append(ϕ)
    return li

In [64]:
a=np.array([[2],[3],[1]])
(a.T).shape

(1, 3)

In [65]:
li=GDA_classif(x,y)

In [66]:
li

[array([[ 2.78611091e-02, -6.24605502e-03,  2.92770604e-02,
          2.07831426e-03,  2.09758179e-03, -5.76842628e-03,
         -3.29680459e-03,  1.88308636e-02, -2.01320808e-02,
          2.67107117e-03, -1.87279755e-03,  1.50858573e-03],
        [-6.24605502e-03,  1.88631898e-02, -2.02439827e-02,
          6.92734173e-04,  1.72158354e-04, -9.28992242e-04,
          3.70449292e-04, -1.09393400e-03,  6.04467427e-03,
         -2.94240067e-03, -2.43079947e-03,  8.61857671e-04],
        [ 2.92770604e-02, -2.02439827e-02,  6.51679114e-02,
          4.11816263e-03,  3.14251941e-03, -3.31674672e-03,
          3.21461086e-03,  1.78007377e-02, -2.31935031e-02,
          5.69728210e-03,  5.30480943e-03,  3.21573683e-03],
        [ 2.07831426e-03,  6.92734173e-04,  4.11816263e-03,
          1.17952907e-02,  5.42993039e-04,  1.86339960e-03,
          3.57486127e-03,  5.86137677e-03, -9.70158547e-04,
          7.17968589e-04,  3.41018649e-03,  8.03568253e-04],
        [ 2.09758179e-03,  1.7215835

In [67]:
li[0].shape

(12, 12)

In [68]:
def Gauss_probab(x,cov_mat,μ):
    """
    x - 1xn
    cov_mat - nxn
    μ0 - nx1
    μ1 - nx1
    """
    n=x.shape[1]
    from math import exp,pi
    import numpy as np
    temp=x.reshape(n,1)-μ
    temp1=(-0.5)*(((temp.T)@np.linalg.inv(cov_mat))@temp)   # temp1 is 1x1 matrix
    temp2=temp1[0][0]
    covmat_det=np.linalg.det(cov_mat)
    out=(exp(temp2))/(((2*pi)**(n/2))*((covmat_det)**(0.5)))
    return out
def GDA_probab(li,x_new):
    """
    li - list [Cov_mat,μ0,μ1,ϕ]
    x_new - 1xn
    """
    px_y0=Gauss_probab(x_new,li[0],li[1])
    px_y1=Gauss_probab(x_new,li[0],li[2])
    ϕ=li[3]
    px=px_y1*ϕ+px_y0*(1-ϕ)
    out=(px_y1*ϕ)/px
    return out

In [69]:
te.shape

(319, 12)

In [70]:
teans.shape

(319, 1)

In [73]:
c=0
for i in range(319):
    x_new=te[i].reshape(1,12)
    temp=GDA_probab(li,x_new)
    if temp<0.5 and teans[i][0]==0:
        c+=1
    elif temp>=0.5 and teans[i][0]==1:
        c+=1
print(c)
acc=c*100/319
print(f'Accuracy={acc}%')

252
Accuracy=78.99686520376176%


In [72]:
y.shape

(1113, 1)

In [75]:
m1=y.shape[0]
y1=y.reshape(m1,)
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(x,y1)
sk_pred=clf.predict(te)

In [77]:
sk_pred.shape

(319,)

In [78]:
c1=0
for i in range(319):
    if (sk_pred[i]==teans[i]):
        c1+=1
print(c1)
acc1=c1*100/319
print(f'Accuracy={acc1}%')

245
Accuracy=76.8025078369906%
